In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

sys.path.insert(0, '..')

In [4]:
import json
from minsearch import Index
import docs

from tqdm.auto import tqdm

In [5]:
import pandas as pd

df_ground_truth = pd.read_csv('../evals/ground_truth_evidently.csv')
ground_truth = df_ground_truth.to_dict(orient='records')

In [7]:
ground_truth[10]

{'question': 'Evidently mapping roles and types',
 'summary_answer': 'The article discusses how to accurately specify roles and types for each column in your dataset to avoid errors in evaluations.',
 'difficulty': 'advanced',
 'intent': 'text',
 'filename': 'docs/library/data_definition.mdx'}

In [6]:
github_data = docs.read_github_data()
parsed_data = docs.parse_data(github_data)


In [8]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

def calculate_num_tokens(search_results):
    json_result = json.dumps(search_results)
    num_tokens = len(encoding.encode(json_result))
    return num_tokens

In [9]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)
                break

    return total_score / len(relevance_total)

In [11]:
def evaluate(
        ground_truth,
        search_function,
        question_column='question',
        id_column='filename'
):
    relevance_total = []
    tokens = []

    for q in ground_truth:
        doc_id = q[id_column]
        results = search_function(q[question_column])
        num_tokens = calculate_num_tokens(results)
        tokens.append(num_tokens)
        relevance = [d[id_column] == doc_id for d in results]
        relevance_total.append(relevance)

    avg_tokens = sum(tokens) / len(tokens)
    
    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
        'num_tokens': avg_tokens
    }

In [ ]:
size = 2000
step = 1000
top_k = 5

def evaluate_params(size, step, top_k):
    chunks = docs.chunk_documents(parsed_data, size=size, step=step)
    
    index = Index(
        text_fields=["content", "filename", "title", "description"],
    )
    
    index.fit(chunks)
    
    def search(query: str):
        return index.search(
            query=query,
            num_results=top_k,
        )
    
    return evaluate(ground_truth, search)

In [13]:
sizes = [1000, 2000, 3000, 5000]
steps = [1000, 2000, 3000]
top_ks = [5, 10, 15]

results = []

for size in sizes:
    for step in steps:
        if step > size:
            continue

        for top_k in top_ks:
            result = evaluate_params(size, step, top_k)
            record = {
                'size': size,
                'step': step,
                'top_k': top_k,
                'hit_rate': result['hit_rate'],
                'num_tokens': result['num_tokens']
            }
            print(record)
            results.append(record)
            

{'size': 1000, 'step': 1000, 'top_k': 5, 'hit_rate': 0.4279918864097363, 'num_tokens': 1351.870182555781}
{'size': 1000, 'step': 1000, 'top_k': 10, 'hit_rate': 0.5294117647058824, 'num_tokens': 2747.5943204868154}
{'size': 1000, 'step': 1000, 'top_k': 15, 'hit_rate': 0.6105476673427992, 'num_tokens': 4089.4624746450304}
{'size': 2000, 'step': 1000, 'top_k': 5, 'hit_rate': 0.43610547667342797, 'num_tokens': 2488.4543610547667}
{'size': 2000, 'step': 1000, 'top_k': 10, 'hit_rate': 0.537525354969574, 'num_tokens': 5075.137931034483}
{'size': 2000, 'step': 1000, 'top_k': 15, 'hit_rate': 0.6308316430020284, 'num_tokens': 7576.789046653144}
{'size': 2000, 'step': 2000, 'top_k': 5, 'hit_rate': 0.4787018255578093, 'num_tokens': 2361.9269776876267}
{'size': 2000, 'step': 2000, 'top_k': 10, 'hit_rate': 0.6064908722109533, 'num_tokens': 4684.156186612576}
{'size': 2000, 'step': 2000, 'top_k': 15, 'hit_rate': 0.6754563894523327, 'num_tokens': 7019.182555780933}
{'size': 3000, 'step': 1000, 'top_k'

In [15]:
import pandas as pd

df = pd.DataFrame(results, columns=['size', 'step', 'top_k', 'hit_rate', 'num_tokens'])

alpha = 2
beta = 0.5
df['score'] = (df.hit_rate ** alpha) / ((df.num_tokens / 1000) ** beta)

df = df.sort_values(by='score', ascending=False)

In [16]:
df.head()

,size,step,top_k,hit_rate,num_tokens,score
2,1000,1000,15,0.610548,4089.462475,0.184334
8,2000,2000,15,0.675456,7019.182556,0.172207
7,2000,2000,10,0.606491,4684.156187,0.169955
1,1000,1000,10,0.529412,2747.594320,0.169087
17,3000,3000,15,0.703854,9399.275862,0.161591


In [14]:
# This represents "retrieval quality adjusted for the cost of processing tokens".
"""

            hit_rate ** alpha
score = ------------------------
        (num_tokens/1000) ** beta
"""

'\n\n            hit_rate ** alpha\nscore = ------------------------\n        (num_tokens/1000) ** beta\n'

In [17]:
import search_agent

In [19]:
config = search_agent.AgentConfig(
    chunk_size=1000,
    chunk_step=1000,
    top_k=15,
)

In [21]:
agent = search_agent.create_agent(config)
agent

Agent(model=OpenAIChatModel(), name='search', end_strategy='early', model_settings=None, output_type=<class 'search_agent.SearchResultArticle'>, instrument=None)

In [22]:
from evals.eval_orchestrator import run_full_evaluation

In [ ]:
await run_full_evaluation(agent, csv_path='../evals/ground_truth_sample_25_2025-10-30-21-03.csv')


=== EVALUATION PIPELINE START ========================================
Start time: 2025-10-30 22:05:35
Configuration:
  Ground truth: ../evals/ground_truth_sample_25_2025-10-30-21-03.csv
  Agent model: gpt-4o-mini
  Judge model: gpt-5-nano
  Max concurrency: 10

=== STEP 1: AGENT EVALUATION =========================================
Loaded 25 ground truth questions
Running agent evaluation...


  0%|          | 0/25 [00:00<?, ?it/s]

forcing output
forcing output
forcing output
